# Mini-Project 2: Oracle for Shor's Algorithm

This notebook implements the following multiplication after taking in two inputs for $a$ and $N$ such that $a<N$ and $n=\lceil\log_{2}{N}\rceil$:

$$U|x\rangle_1|y\rangle_n = 
\begin{cases} 
|x\rangle_1|a y \mod N\rangle_n & \text{if } x=1 \\
|x\rangle_1|y\rangle_n & \text{if } x=0
\end{cases}$$

In this next cell, I define the function modular_mult_oracle which satisfies the requirements above:

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.circuit.library import QFT
from qiskit.quantum_info import Statevector
import numpy as np

def modular_mult_oracle(a, N):
    n = int(np.ceil(np.log2(N))) 
    
    x = QuantumRegister(1, 'x')         
    y = QuantumRegister(n, 'y')          
    anc = QuantumRegister(2, 'anc')      
    qc = QuantumCircuit(x, y, anc)

    qc.x(y)
    qc.mcx(list(y) + [x[0]], anc[0])    
    qc.x(y)

    qft = QFT(n, do_swaps=False)
    qc.append(qft, y)

    for i in range(n):
        angle = 2 * np.pi * (a * (2 ** i)) / (2 ** n)
        qc.cp(angle, anc[0], y[i])      

    iqft = QFT(n, do_swaps=False, inverse=True)
    qc.append(iqft, y)

    qc.x(y)
    qc.mcx(list(y) + [x[0]], anc[0])
    qc.x(y)

    return qc

In this next cell, I run the function modular_mult_oracle to print out the circuit for $a=7$ and $N=15$ (this should still work for other values of N):

In [ ]:
oracle=modular_mult_oracle(a,N)
print(f"Circuit for a=7, N=15 ({oracle.num_qubits} qubits):")
oracle.draw(fold=-1)

In these next cell, I print out the circuit depth, the total qubit count, and the gate count.

In [ ]:
# Number of gates
print("Gate counts:")
oracle.count_ops()

In [ ]:
# Gate depth
print("\nCircuit depth:")
oracle.depth()

In [ ]:
# Number of qubits
print("\nTotal qubits:")
oracle.num_qubits

In this next cell, I run a demo for a=7 and for y=5 and y=15, with N=15

In [ ]:
# Demo for N=15
def demo_oracle(a, N, y_values):
    n = int(np.ceil(np.log2(N)))
    oracle = modular_mult_oracle(a, N)
    
    for y in y_values:
        x_reg = QuantumRegister(1, 'x')
        y_reg = QuantumRegister(n, 'y')
        anc_reg = QuantumRegister(2, 'anc')
        qc = QuantumCircuit(x_reg, y_reg, anc_reg)
        
        qc.x(x_reg) 
        y_bin = format(y, f'0{n}b')
        for i, bit in enumerate(reversed(y_bin)):
            if bit == '1':
                qc.x(y_reg[i])
        
        qc.compose(oracle, inplace=True)
        
        state = Statevector(qc)
        state_dict = state.probabilities_dict()  
        
        print(f"\nInput y = {y} ({y_bin}):")
        if y < N:
            expected = (a * y) % N
            expected_bin = format(expected, f'0{n}b')
            print(f"Expected: |1⟩|{expected_bin}⟩|00⟩ ({a}*{y} mod {N} = {expected})")
        else:
            print(f"Expected: |1⟩|{y_bin}⟩|00⟩ (unchanged)")
        
        print("Output state (non-zero amplitudes):")
        for basis_state, prob in state_dict.items():
            if not np.isclose(prob, 0):
                print(f"  |{basis_state}⟩: amplitude = {np.sqrt(prob):.4f}")

N = 15
a = 7
test_values = [5, 15] 
demo_oracle(a, N, test_values)

The main source used was Brauregard (2003)